# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms)
kgrid = [1, 1, 1]
Ecut = 5
basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (λ=λ, X=X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.

In [4]:
struct MyMixing
    α  # Damping parameter
end
MyMixing() = MyMixing(0.7)

function DFTK.mix(mixing::MyMixing, basis, ρin::RealFourierArray, ρout::RealFourierArray; n_iter, kwargs...)
    if n_iter <= 2
        # Just do simple mixing
        ρin + mixing.α * (ρout - ρin)
    else
        # Use the KerkerMixing from DFTK
        DFTK.mix(KerkerMixing(α=mixing.α), basis, ρin, ρout; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            ρout-ρin   Eₙ₋₁-Eₙ    Diag
---   ---------------   --------   --------   ----
1     -7.195651494858   3.43e-01        NaN   0.0 
2     -7.242140528487   1.67e-01   4.65e-02   0.0 
3     -7.248221006820   7.24e-02   6.08e-03   0.0 
4     -7.248906558405   3.96e-02   6.86e-04   0.0 
5     -7.249120644687   2.19e-02   2.14e-04   0.0 
6     -7.249186599115   1.23e-02   6.60e-05   0.0 
7     -7.249207108734   6.98e-03   2.05e-05   0.0 
8     -7.249213630304   4.03e-03   6.52e-06   0.0 
9     -7.249215767102   2.36e-03   2.14e-06   0.0 
10    -7.249216491414   1.41e-03   7.24e-07   0.0 
11    -7.249216745660   8.49e-04   2.54e-07   0.0 
12    -7.249216837938   5.19e-04   9.23e-08   0.0 
13    -7.249216872456   3.20e-04   3.45e-08   0.0 
14    -7.249216885708   2.00e-04   1.33e-08   0.0 
15    -7.249216890906   1.25e-04   5.20e-09   0.0 


Note that the default convergence criterion is on the difference of
energy from one step to the other; when this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixpoint
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.